In [23]:
import pandas as pd
from os import listdir

full_path = "/Users/josephdespres/Documents/Python/pythonChallenges/data_cleaning_proj/first-midterm-despresj/data/"

files = listdir("data")
pd.DataFrame(files)
# incidents
gtd = pd.read_csv(full_path + files[0], encoding='latin1')
# county-city-poverty-data
poverty = pd.read_csv(full_path + files[1])
cities_pop = pd.read_csv(full_path + files[2], encoding='latin1')
county_cities = pd.read_csv(full_path + files[3], encoding='latin1')


In [24]:
cities_pop

,SUMLEV,STATE,COUNTY,PLACE,NAME,STNAME,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,Alabama,Alabama,4829479,4843214,4853875,4863300
1,162,1,0,124,Abbeville city,Alabama,2631,2619,2616,2603
2,162,1,0,460,Adamsville city,Alabama,4447,4428,4395,4360
3,162,1,0,484,Addison town,Alabama,748,747,740,738
4,162,1,0,676,Akron town,Alabama,342,337,337,334
...,...,...,...,...,...,...,...,...,...,...
81580,157,56,43,99990,Balance of Washakie County,Wyoming,2744,2697,2704,2669
81581,50,56,45,0,Weston County,Wyoming,7163,7179,7230,7236
81582,157,56,45,56215,Newcastle city,Wyoming,3496,3505,3525,3535
81583,157,56,45,79125,Upton town,Wyoming,1094,1101,1107,1106


# gameplan
here we have 3 datasets one on poverty by county, one on population by city, one that will bridge the two. 
Remember cities and counties are written differently (eg st. louis could be saint louis). The goal here will be no missing data.

- make df: `popcounty` by adding `counties` to population-by-city `cities_pop`.
- make df: `povcity` by joining poverty-by-county `cities_pop` and adding cities `county_cities` 
- make df: stats_city_county by combining `povcity` and `popcounty`
- make df: joining this to incident.

In [27]:
cities_pop

,SUMLEV,STATE,COUNTY,PLACE,NAME,STNAME,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,Alabama,Alabama,4829479,4843214,4853875,4863300
1,162,1,0,124,Abbeville city,Alabama,2631,2619,2616,2603
2,162,1,0,460,Adamsville city,Alabama,4447,4428,4395,4360
3,162,1,0,484,Addison town,Alabama,748,747,740,738
4,162,1,0,676,Akron town,Alabama,342,337,337,334
...,...,...,...,...,...,...,...,...,...,...
81580,157,56,43,99990,Balance of Washakie County,Wyoming,2744,2697,2704,2669
81581,50,56,45,0,Weston County,Wyoming,7163,7179,7230,7236
81582,157,56,45,56215,Newcastle city,Wyoming,3496,3505,3525,3535
81583,157,56,45,79125,Upton town,Wyoming,1094,1101,1107,1106


In [6]:
poverty

,State,Area_Name,PCTPOVALL_2015,PCTPOV017_2015,MEDHHINC_2015
0,AL,Alabama,18.5,26.5,"44,833"
1,AL,Autauga County,12.7,18.8,"56,580"
2,AL,Baldwin County,12.9,19.6,"52,387"
3,AL,Barbour County,32.0,45.2,"31,433"
4,AL,Bibb County,22.2,29.3,"40,767"
...,...,...,...,...,...
3188,WY,Sweetwater County,8.5,10.2,"71,867"
3189,WY,Teton County,6.6,7.6,"83,290"
3190,WY,Uinta County,9.8,11.9,"62,968"
3191,WY,Washakie County,11.2,15.7,"56,088"


In [7]:
county_cities

,latitude,longitude,city,state,county
0,40.922326,-72.637078,Holtsville,NY,Suffolk
1,40.922326,-72.637078,Holtsville,NY,Suffolk
2,18.165273,-66.722583,Adjuntas,PR,Adjuntas
3,18.393103,-67.180953,Aguada,PR,Aguada
4,18.455913,-67.145780,Aguadilla,PR,Aguadilla
...,...,...,...,...,...
42736,55.094325,-131.566827,Metlakatla,AK,Prince Wales Ketchikan
42737,55.517921,-132.003244,Point Baker,AK,Prince Wales Ketchikan
42738,55.395359,-131.675370,Ward Cove,AK,Ketchikan Gateway
42739,56.449893,-132.364407,Wrangell,AK,Wrangell Petersburg


,SUMLEV,STATE,COUNTY,PLACE,NAME,STNAME,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,Alabama,Alabama,4829479,4843214,4853875,4863300
1,162,1,0,124,Abbeville city,Alabama,2631,2619,2616,2603
2,162,1,0,460,Adamsville city,Alabama,4447,4428,4395,4360
3,162,1,0,484,Addison town,Alabama,748,747,740,738
4,162,1,0,676,Akron town,Alabama,342,337,337,334
...,...,...,...,...,...,...,...,...,...,...
81580,157,56,43,99990,Balance of Washakie County,Wyoming,2744,2697,2704,2669
81581,50,56,45,0,Weston County,Wyoming,7163,7179,7230,7236
81582,157,56,45,56215,Newcastle city,Wyoming,3496,3505,3525,3535
81583,157,56,45,79125,Upton town,Wyoming,1094,1101,1107,1106
